In [1]:


import xml.etree.cElementTree as ET
from collections import defaultdict 
import re

osm_file = open("/Users/Moscalu/Desktop/Python/Lesson_4/bronx_new-york.osm.xml")

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 
tagbook = dict()

def tags(file):
    for event, elem in ET.iterparse(file, events=('start',)):
        m = elem.tag
        if m in tagbook: 
            tagbook[m] +=1  
        else:
            tagbook[m] = 1   
    return tagbook

print_sorted_dict(tags(osm_file))

bounds: 1
member: 6517
nd: 1046244
node: 833276
osm: 1
relation: 596
tag: 793467
way: 131207


In [2]:
import pprint
osm_file = open("/Users/Moscalu/Desktop/Python/Lesson_4/bronx_new-york.osm.xml")
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()

        street_types[street_type] += 1

def print_sorted_dict(d):
    keys = d.keys()
    keys = sorted(keys, key=lambda s: s.lower())
    for k in keys:
        v = d[k]
        print "%s: %d" % (k, v) 

def is_street_name(elem):
    return (elem.tag == "tag") and (elem.attrib['k'] == "addr:street")

def audit():
    for event, elem in ET.iterparse(osm_file):
        if is_street_name(elem):
            audit_street_type(street_types, elem.attrib['v'])    
    print_sorted_dict(street_types)    




In [158]:
osm_file = open("/Users/Moscalu/Desktop/Python/Lesson_4/bronx_new-york.osm.xml","r")
street_type_re = re.compile(r'\S+\.?$', re.IGNORECASE)
street_types = defaultdict(set)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Shores", "Highway"]
mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "Rd" : "Road",
            "Rd." : "Road",
            "Blvd" : "Boulevard",
            "Blvd." : "Boulevard",
            "Cir" : "Circle",
            "Cir." : "Circle",
            "Ct" : "Court",
            "Ct." : "Court",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Pl" : "Place",
            "Pl." : "Place",
            }
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        
        if street_type not in expected:
           
            street_types[street_type].add(street_name)
    


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit():
    counter = 0
    for event, elem in ET.iterparse(osm_file, events=('start',)):
        if elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])    
       
                counter +=1 
                if counter == 10:
                    break 

def update_name(name, mapping):
    m = street_type_re.search(name)
    if m not in expected:
            if m.group() in mapping.keys():
                name = re.sub(m.group(), mapping[m.group()], name)

    return name
    pprint.pprint(dict(street_types))



In [157]:
audit()

In [62]:
#tag types
osm_file = open("/Users/Moscalu/Desktop/Python/Lesson_4/bronx_new-york.osm.xml","r")
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

def key_type(element, keys):
    if (element.tag == "tag"):
        lower_re = lower.search(element.attrib['k'])
        lower_colon_re = lower_colon.search(element.attrib['k'])
        problemchars_re = problemchars.search(element.attrib['k'])
        if lower_re:
            keys["lower"] +=1
        elif lower_colon_re:
            keys["lower_colon"] +=1
        elif problemchars_re:
            keys ["problemchars"] +=1
        else:
            keys ['other'] +=1
        pass
        
    return keys


def process_map(filename):
    counter =0
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for event, element in ET.iterparse(osm_file, events=('start',)):
        counter +=1
        keys = key_type(element, keys)
        if counter == 50:
            break
    return keys


In [63]:
process_map(osm_file)

{'lower': 4, 'lower_colon': 0, 'other': 0, 'problemchars': 0}

In [154]:
osm_file = open("/Users/Moscalu/Desktop/Python/Lesson_4/bronx_new-york.osm.xml","r")
def get_user(element):
    if 'uid' in element.attrib:
         return element.attrib['uid']


def process_map(filename):
    users = set()
    counter =0
    for _, element in ET.iterparse(filename):
        
        if counter == 1000:
                break
        counter +=1
        users.add(get_user(element))
    return users

In [155]:
process_map(osm_file)

{None,
 '117975',
 '123364',
 '1240849',
 '1376118',
 '1424765',
 '14293',
 '147510',
 '1764427',
 '1764514',
 '1781294',
 '182051',
 '19239',
 '193146',
 '197756',
 '2012449',
 '2065004',
 '207745',
 '2359041',
 '2421207',
 '252811',
 '2591427',
 '259657',
 '273930',
 '284217',
 '30521',
 '3158',
 '321862',
 '322785',
 '336460',
 '35195',
 '354704',
 '38533',
 '3962',
 '439046',
 '567237',
 '624003',
 '689176',
 '716239',
 '741163',
 '840447'}

In [163]:
osm_file = open("/Users/Moscalu/Desktop/Python/Lesson_4/bronx_new-york.osm.xml","r")
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Shores", "Highway", "Circle","Loop", "Terrace", "North", "South", "West",
            "East"]

mapping = { "St": "Street",
            "St.": "Street",
            "Ave" : "Avenue",
            "Ave." : "Avenue",
            "Rd" : "Road",
            "Rd." : "Road",
            "Blvd" : "Boulevard",
            "Blvd." : "Boulevard",
            "Cir" : "Circle",
            "Cir." : "Circle",
            "Ct" : "Court",
            "Ct." : "Court",
            "Dr" : "Drive",
            "Dr." : "Drive",
            "Pl" : "Place",
            "Pl." : "Place",
            "Pkwy" : "Parkway"
            }
def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)
            
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

def update_name(name, mapping):
    
        m = street_type_re.search(name)
        if m not in expected:
             if m.group() in mapping.keys():
                name = re.sub(m.group(), mapping[m.group()], name)
    
        return name
    
st_types = audit(osm_file)
st_types

defaultdict(set,
            {'1': {'Grand Concourse #1'},
             'Ave': {'Riverdale Ave'},
             'Ave.': {'Britton St. bet. Olinville Ave. and Barker Ave.'},
             'Bridge': {'Harlem River Park Bridge'},
             'Broadway': {'Broadway'},
             'Bush': {'Ploughmans Bush'},
             'Camp': {'Pennyfield Camp'},
             'Concourse': {'Grand Concourse'},
             'Crescent': {'Albany Crescent',
              'Brian Crescent',
              'Knolls Crescent',
              'Park Crescent',
              'Point Crescent',
              'Saint Georges Crescent'},
             'Esplanade': {'Esplanade'},
             'Estate': {'Glenz Estate'},
             'Expressway': {'Bruckner Expressway',
              'Cross Bronx Expressway',
              'Major Deegan Expressway'},
             'Hill': {'Fort George Hill'},
             'Island': {'Randalls Island', 'Wards Island'},
             'John': {'Avenue Saint John'},
             'Malba': {'Cente

In [6]:
schema = {
    'node': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'lat': {'required': True, 'type': 'float', 'coerce': float},
            'lon': {'required': True, 'type': 'float', 'coerce': float},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'node_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string'}
            }
        }
    },
    'way': {
        'type': 'dict',
        'schema': {
            'id': {'required': True, 'type': 'integer', 'coerce': int},
            'user': {'required': True, 'type': 'string'},
            'uid': {'required': True, 'type': 'integer', 'coerce': int},
            'version': {'required': True, 'type': 'string'},
            'changeset': {'required': True, 'type': 'integer', 'coerce': int},
            'timestamp': {'required': True, 'type': 'string'}
        }
    },
    'way_nodes': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'node_id': {'required': True, 'type': 'integer', 'coerce': int},
                'position': {'required': True, 'type': 'integer', 'coerce': int}
            }
        }
    },
    'way_tags': {
        'type': 'list',
        'schema': {
            'type': 'dict',
            'schema': {
                'id': {'required': True, 'type': 'integer', 'coerce': int},
                'key': {'required': True, 'type': 'string'},
                'value': {'required': True, 'type': 'string'},
                'type': {'required': True, 'type': 'string', 'required': True}
            }
        }
    }
}





import csv
import codecs
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

OSM_PATH = ("/Users/Moscalu/Desktop/Python/Lesson_4/bronx_new-york.osm.xml","r")

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']


def correct_k(k):
    index=k.find(':')
    typ=k[:index]
    k=k[index+1:]    
    return k,typ

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""
    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []  # Handle secondary tags the same way for both node and way elements

    if element.tag=='node':
        for i in node_attr_fields:
            node_attribs[i]=element.attrib[i]
            
    if element.tag=='way':
        for i in way_attr_fields:
            way_attribs[i]=element.attrib[i]
        
    for tag in element.iter("tag"):
        dic={}
        attributes=tag.attrib
        if problem_chars.search(tag.attrib['k']):
            continue
        
        if element.tag=='node':
            dic['id']=node_attribs['id']
        else:
            dic['id']=way_attribs['id']
        
        dic['value'] = attributes['v']

        colon_k=LOWER_COLON.search(tag.attrib['k'])
        if colon_k:
            print colon_k.group(0)
            print tag.attrib['k']
            dic['key'],dic['type']=correct_k(tag.attrib['k'])
        else:
            dic['key']=attributes['k']
            dic['type']='regular'
        
        
        #print dic
        tags.append(dic)
    
    if element.tag=='way':
        position=0
        for nd in element.iter("nd"):
            way_node_dic={}
            way_node_dic['id']=way_attribs['id']
            way_node_dic['node_id']=nd.attrib['ref']
            way_node_dic['position']=position
            position = position + 1
            way_nodes.append(way_node_dic)
             
    if element.tag == 'node':
        return {'node': node_attribs, 'node_tags': tags}
    elif element.tag == 'way':
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}
# ================================================== #
#               Helper Functions                     #
# ================================================== #
def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()


def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_strings = (
            "{0}: {1}".format(k, v if isinstance(v, str) else ", ".join(v))
            for k, v in errors.iteritems()
        )
        raise cerberus.ValidationError(
            message_string.format(field, "\n".join(error_strings))
        )


class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)


# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
        codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
                elif element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

AttributeError: 'module' object has no attribute 'schema'